##Importing 라이브러리

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42)  # 무조건 42로 세팅!!

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

##파일 불러오기

In [ ]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
submission = pd.read_csv("/content/sample_submission.csv")
print(train.shape, test.shape, submission.shape)

(48199, 13) (7695, 12) (7695, 2)


##데이터 전처리 (Train + Test)

###0)불필요 열 제거

In [ ]:
train.drop(columns=['click_id', 'session ID'], inplace=True)

In [ ]:
test.drop(columns=['click_id', 'session ID'], inplace=True)

###1) main category 결측치 채우기

In [ ]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (train["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
train.loc[mask, "page 1 (main category)"] = train.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(train.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])

31


In [ ]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (test["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
test.loc[mask, "page 1 (main category)"] = test.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(test.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])

3


###1-1)clothing model 결측치 채우기

In [ ]:
# clothing model의 최빈값 찾기 (동일한 특성을 가진 데이터 그룹별)
mode_values = train.groupby(['page 1 (main category)', 'location', 'colour', 'model photography', 'page'])['page 2 (clothing model)'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()
mode_values.columns = ['page 1 (main category)', 'location', 'colour', 'model photography', 'page', 'clothing_model_mode']

# Train & Test 데이터에 적용
train = train.merge(mode_values, on=['page 1 (main category)', 'location', 'colour', 'model photography', 'page'], how='left')
test = test.merge(mode_values, on=['page 1 (main category)', 'location', 'colour', 'model photography', 'page'], how='left')

# 결측치 채우기
train.loc[train['page 2 (clothing model)'].isna(), 'page 2 (clothing model)'] = train['clothing_model_mode']
test.loc[test['page 2 (clothing model)'].isna(), 'page 2 (clothing model)'] = test['clothing_model_mode']

# 임시 컬럼 삭제
train.drop(columns=['clothing_model_mode'], inplace=True)
test.drop(columns=['clothing_model_mode'], inplace=True)

###2)시간과 관련한 데이터 추출

In [ ]:
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['weekday'] = train['date'].dt.weekday
train.drop(columns=['date'], inplace=True)

In [ ]:
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.weekday
test.drop(columns=['date'], inplace=True)

###3)colour

In [ ]:
# 색상 그룹 매핑 (높은 Click 비율 → 낮은 숫자)
color_map = {
    2: 0, 3: 0, 8: 0,  # 가장 높은 Click 비율 (가장 낮은 숫자)
    4: 1, 7: 1, 14: 1,  # 두 번째 그룹
    9: 2, 10: 2, 11: 2, 13: 2,  # 세 번째 그룹
    1: 3, 5: 3, 6: 3, 12: 3  # 가장 낮은 Click 비율 (가장 높은 숫자)
}

# colour 컬럼 변환
train["colour"] = train["colour"].map(color_map)

In [ ]:
test["colour"] = test["colour"].map(color_map)

###country target encoding

In [ ]:
train["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

In [ ]:
test["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

###5)price log변환

In [ ]:
train['log_price'] = np.log1p(train['price'])

In [ ]:
test['log_price'] = np.log1p(test['price'])

###6)상품별 인기 지표

In [ ]:
product_ctr = train.groupby('page 2 (clothing model)')['Clicked'].mean().reset_index()
product_ctr.columns = ['page 2 (clothing model)', 'product_ctr']

train = train.merge(product_ctr, on='page 2 (clothing model)', how='left')

In [ ]:
test = test.merge(product_ctr, on='page 2 (clothing model)', how='left')
# Test 데이터에서 product_ctr가 없는 경우, Train의 평균 CTR로 대체
test['product_ctr'].fillna(train['product_ctr'].mean(), inplace=True)

###7)main category & price 관계

In [ ]:
# 카테고리별 평균 가격 계산
category_price_mean = train.groupby('page 1 (main category)')['price'].mean().reset_index()
category_price_mean.columns = ['page 1 (main category)', 'category_avg_price']

# 제품의 가격 대비 카테고리 평균 가격 비율 계산
train = train.merge(category_price_mean, on='page 1 (main category)', how='left')
train['category_price_ratio'] = train['price'] / train['category_avg_price']

In [ ]:
# 카테고리별 평균 가격 계산
category_price_mean = test.groupby('page 1 (main category)')['price'].mean().reset_index()
category_price_mean.columns = ['page 1 (main category)', 'category_avg_price']

# 제품의 가격 대비 카테고리 평균 가격 비율 계산
test = test.merge(category_price_mean, on='page 1 (main category)', how='left')
test['category_price_ratio'] = test['price'] / test['category_avg_price']

###7)같은 모델인데 가격이 다른 경우

In [ ]:
# clothing model별 평균 가격 계산
clothing_price_mean = train.groupby('page 2 (clothing model)')['price'].mean().reset_index()
clothing_price_mean.columns = ['page 2 (clothing model)', 'clothing_avg_price']

# 개별 제품의 가격과 평균 가격 차이 계산
train = train.merge(clothing_price_mean, on='page 2 (clothing model)', how='left')
train['price_anomaly'] = train['price'] - train['clothing_avg_price']
train['price_anomaly_ratio'] = train['price'] / train['clothing_avg_price']

#page 2 (clothing model) 삭제
train.drop(columns=['page 2 (clothing model)', 'price'], inplace=True)

In [ ]:
# clothing model별 평균 가격 계산
clothing_price_mean = test.groupby('page 2 (clothing model)')['price'].mean().reset_index()
clothing_price_mean.columns = ['page 2 (clothing model)', 'clothing_avg_price']

# 개별 제품의 가격과 평균 가격 차이 계산
test = test.merge(clothing_price_mean, on='page 2 (clothing model)', how='left')
test['price_anomaly'] = test['price'] - test['clothing_avg_price']
test['price_anomaly_ratio'] = test['price'] / test['clothing_avg_price']

#page 2 (clothing model) 삭제
test.drop(columns=['page 2 (clothing model)', 'price'], inplace=True)

###8)중복되는 데이터 삭제

In [ ]:
train.drop(columns=['country','year', 'month'], inplace=True)

In [ ]:
test.drop(columns=['country', 'year', 'month'], inplace=True)

##학습하기
-XGboost 모델 사용하기
-하이퍼파라미터의 영향을 많이 받는 모델이기에 최적의 조합 찾기

In [ ]:
X = train.drop('Clicked', axis=1)
y = train['Clicked']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(38559, 16) (9640, 16) (38559,) (9640,)


-

In [ ]:
clf=XGBClassifier()
params = {'n_estimators':[50, 100, 200],
          'max_depth':[2,3,4],
          }
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 2, 'n_estimators': 50}
최고 예측 정확도: 0.6822


In [ ]:
xgb_clf = XGBClassifier(**grid_cv.best_params_)
#[0.6844-0.6839]
#xgb_clf = XGBClassifier(n_estimators=100, max_depth=4, learning_rate = 0.05)
xgb_clf.fit(X_train, y_train)

train_pred = xgb_clf.predict(X_train)
val_pred = xgb_clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, train_pred))
print("Val ACC : %.4f" % accuracy_score(y_val, val_pred))

Train ACC : 0.6853
Val ACC : 0.6831


In [ ]:
xgb_clf = XGBClassifier(n_estimators=148, max_depth=2, learning_rate = 0.2)
xgb_clf.fit(X_train, y_train)

#148[0.6874-0.6845]
train_pred = xgb_clf.predict(X_train)
val_pred = xgb_clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, train_pred))
print("Val ACC : %.4f" % accuracy_score(y_val, val_pred))

Train ACC : 0.6875
Val ACC : 0.6857


In [ ]:
feature_importance = pd.DataFrame(xgb_clf.feature_importances_.reshape((1, -1)), columns=X_train.columns, index=['feature_importance'])
feature_importance

order  page 1 (main category)   colour  location  \
feature_importance  0.130852                0.058253  0.02463  0.012223   

                    model photography      page       day   weekday  \
feature_importance           0.002707  0.024132  0.014012  0.019689   

                    country_encoded  log_price  product_ctr  \
feature_importance         0.015028    0.04147     0.595879   

                    category_avg_price  category_price_ratio  \
feature_importance                 0.0              0.018031   

                    clothing_avg_price  price_anomaly  price_anomaly_ratio  
feature_importance            0.016432       0.012669             0.013992

##submission.csv 생성

In [ ]:
X_test = test

output = xgb_clf.predict(X_test)
assert len(output) == 7695  # sanity check

In [ ]:
submission['Clicked'] = output
submission.to_csv('submission.csv', index=False)